In [2]:
import torch
import torch.nn as nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt


In [3]:
transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)])
train_path="E:/Kaggle/imagenet-mini/imagenet-mini/train"
test_path="E:/Kaggle/imagenet-mini/imagenet-mini/val"
train_dataset=datasets.ImageFolder(train_path,transform=transform)
test_dataset=datasets.ImageFolder(test_path,transform=transform)
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=True)


In [4]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.net=nn.Sequential(
            nn.Conv2d(3,96,kernel_size=11,stride=4,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),
            nn.Conv2d(96,256,kernel_size=5,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),
            nn.Conv2d(256,384,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(384,384,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(384,256,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),
            nn.Flatten(),
            nn.Linear(9216,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,1000),            
        )
    def forward(self,x):
        return self.net(x)
        

In [5]:
model=AlexNet()
model=model.cuda()
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.08,momentum=0.9)


In [6]:
epochs=10
losses=[]
for epoch in range(epochs):
    correct,total=0,0
    for (x,y) in train_loader:  ##或者写for i,(x,y) in enumerate(dataloader)
        x,y=x.cuda(),y.cuda()
        pred=model(x)
        loss=loss_fn(pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        _,predicted=torch.max(pred,dim=1)
        correct+=torch.sum(predicted==y)
        total+=y.size(0)
    accuracy=100*correct/total
    print(f"Epoch[{epoch+1}/{epochs}],Loss:{loss.item():.4f},Accuracy: {accuracy:.2f}%")
fig=plt.figure(figsize=(10,8))
plt.plot(range(len(losses)),losses)
plt.show()


Epoch[1/10],Loss:6.9257,Accuracy: 0.29%
Epoch[2/10],Loss:6.8357,Accuracy: 0.34%


KeyboardInterrupt: 

In [ ]:
##torch.max返回两个张量，第一个是最大值，第二个是对应的索引
##比如说pred=torch.tensor([
#                     [0.8,0.1,0.1],
#                     [0.2,0.7,0.1],
#                     [0.3,0.3,0.4],
#                     [0.1,0.2,0.7]])
#真实标签y: y=torch.tensor([0,1,2,2])
#即可以认为模型正确预测了4个样本的标签

In [ ]:
correct,total=0,0
with torch.no_grad():
  for (x,y) in test_loader:
      x,y=x.cuda(),y.cuda()
      pred=model(x)
      _,predicted=torch.max(pred,dim=1)
      correct+=torch.sum(predicted==y)
      total+=y.size(0)
print(f"预测精度为:{100*correct/total:.2f}%")
    